In [7]:
import re
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## Helpers

In [8]:
def get_query_jsn(url):
    """returns json from request
    -INPUT:
        -url: str of url to be requested
    -OUTPUT:
        - list of päevakorrad"""
    return json.loads(requests.get(url).content)

In [9]:
def get_soup(link):
    """returns beautifulsoup object from link content
    -INPUT:
        -link: str of url to be requested
    -OUTPUT:
        - beautifulsoup of link content"""
    soup=BeautifulSoup(requests.get(link).content)
    return soup

In [10]:
#historic faction data is in html
def get_soup_faction_members(soup):
    """function to get historic faction members information
    -INPUT:
        -soup: beautifulsoup ob ject of page where faction member info is
    -OUTPUT:
        -list of faction members information"""
    factions=[]
    faction_name=soup.find('h3').text
    faction_members=[]
    for i, tag in enumerate(soup.find('h3').next_siblings):
        if tag.name=='h3' or tag.name=='h2':
            faction_name=tag.text
            factions.extend(faction_members)
            faction_members=[]
        #esimehed, aseesimehed
        if tag.name=='p':
            text=tag.text
            text=text.replace('Aseesimees: ', '').replace('Esimees: ', '').replace('Aseesimehed: ', '').replace('Esimehed: ', '').replace('\xa0', ' ')
            names=text.split(', ')
            for name in names:
                for subname in name.split('\n'):
                    #usually names not so long
                    if len(subname.split())<10 and subname!='':
                        faction_members.append((subname, faction_name))
        #list of faction members
        if tag.name=='ul':
            list_members=[(li.text, faction_name) for li in tag.findAll('li')]
            faction_members.extend(list_members)

    factions.extend(faction_members)
    return factions

In [11]:
def get_koosseis_faction_members(koosseisud=['xi', 'xii', 'xiii']):
    """get faction members from Riigikogu koosseisud
    -INPUT:
        -koosseisud:  list of str of koosseis number
    -OUTPIT:
        -pd.DataFrame of faction members information"""
    df_factions=pd.DataFrame()
    for koosseis in koosseisud:
        print(f'working in faction {koosseis}')
        soup=get_soup(f'https://www.riigikogu.ee/tutvustus-ja-ajalugu/riigikogu-ajalugu/{koosseis}-riigikogu-koosseis/fraktsioonid/')
        faction_members=get_soup_faction_members(soup)
        df_=pd.DataFrame(faction_members)
        df_.columns=['name', 'faction']
        df_['koosseis']=koosseis
        df_factions=df_factions.append(df_)
    df_factions=df_factions.drop_duplicates()
    df_factions=df_factions.reset_index(drop=True)
    return df_factions

## Get all historic factions from html pages

In [12]:
df_factions=get_koosseis_faction_members()

working in faction xi
working in faction xii
working in faction xiii


In [13]:
df_factions.shape

(305, 3)

In [14]:
df_factions.faction.value_counts()

Eesti Reformierakonna fraktsioon                    89
Eesti Keskerakonna fraktsioon                       72
Sotsiaaldemokraatliku Erakonna fraktsioon           42
Isamaa ja Res Publica Liidu fraktsioon              41
Fraktsioonidesse mittekuulunud Riigikogu liikmed    16
Isamaa fraktsioon                                   11
Fraktsiooni mittekuuluvad saadikud                   8
Eestimaa Rahvaliidu fraktsioon                       7
Eesti Konservatiivse Rahvaerakonna fraktsioon        7
Erakonna Eestimaa Rohelised fraktsioon               6
Eesti Vabaerakonna fraktsioon                        6
Name: faction, dtype: int64

In [15]:
df_factions.head()

,name,faction,koosseis
0,Mai Treial (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi
1,Villu Reiljan (02.04.2007–26.05.2010),Eestimaa Rahvaliidu fraktsioon,xi
2,Karel Rüütli (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi
3,Ester Tuiksoo (09.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi
4,Tarmo Mänd (03.04.2007–12.03.2010),Eestimaa Rahvaliidu fraktsioon,xi


## Get current koosseis using API

In [16]:
current_koosseis=get_query_jsn('https://api.riigikogu.ee/api/plenary-members?lang=et')

In [17]:
df_current_faction=pd.DataFrame([[i['fullName'], i['faction']['name']] for i in current_koosseis])
df_current_faction.columns=['name', 'faction']
df_current_faction['koosseis']='xiv'
df_current_faction.head()

,name,faction,koosseis
0,Jaak Aab,Eesti Keskerakonna fraktsioon,xiv
1,Merry Aart,Eesti Konservatiivse Rahvaerakonna fraktsioon,xiv
2,Taavi Aas,Eesti Keskerakonna fraktsioon,xiv
3,Annely Akkermann,Eesti Reformierakonna fraktsioon,xiv
4,Yoko Alender,Eesti Reformierakonna fraktsioon,xiv


## Append current and historic data

In [18]:
df_factions_all=df_factions.append(df_current_faction)
df_factions_all=df_factions_all.reset_index(drop=True)
df_factions_all.shape

(406, 3)

## Do some cleaning

In [19]:
df_factions_all['name'].nunique(), df_factions_all['faction'].nunique()

(281, 12)

In [20]:
#a bit of cleaning and formatting
df_factions_all['name']=df_factions_all.name.str.strip()
df_factions_all['faction']=df_factions_all.faction.str.strip()
df_factions_all['name'].nunique(), df_factions_all['faction'].nunique()

(280, 12)

## Get times (if from or to exists) if person has changed faction during koosseis

In [21]:
def get_from_to(text):
    """extract from and to in faction member nae field"""
    if ' (alates' in text.lower():
        return text.split(' (alates')[-1].replace(')', '').strip(), None
    start_end=re.findall('\(\d\d\.\d\d\.\d\d\d\d–\d\d\.\d\d\.\d\d\d\d\)', text)
    if len(start_end)>0:
        start, end = start_end[0].split('–')
        return start.replace('(', '').strip(), end.replace(')', '').strip()
    return None, None

In [22]:
df_factions_all['from'], df_factions_all['to']= zip(*df_factions_all['name'].map(get_from_to))

In [23]:
df_factions_all['from'].value_counts()

07.07.2010    5
02.04.2007    4
09.04.2007    1
03.04.2007    1
31.05.2010    1
22.04.2008    1
12.03.2010    1
07.12.2010    1
Name: from, dtype: int64

## Fill members from and to based on koosseis start time

In [24]:
#this was taken from wikipedia
koosseis_start={'xi':'02.04.2007',
                'xii':'04.04.2011',
                'xiii':'30.03.2015',
                'xiv':'04.04.2019'}

koosseis_end={'xi':'03.04.2011',
                'xii':'29.03.2015',
                'xiii':'03.04.2019',
                'xiv':'03.04.2023'}

In [26]:
df_factions_all['from']=np.where(pd.isnull(df_factions_all['from']), df_factions_all.koosseis.apply(lambda x: koosseis_start[x]), df_factions_all['from'])
df_factions_all['to']=np.where(pd.isnull(df_factions_all['to']), df_factions_all.koosseis.apply(lambda x: koosseis_end[x]), df_factions_all['to'])
df_factions_all['from']=pd.to_datetime(df_factions_all['from'])
df_factions_all['to']=pd.to_datetime(df_factions_all['to'])

In [27]:
df_factions_all

,name,faction,koosseis,from,to
0,Mai Treial (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07
1,Villu Reiljan (02.04.2007–26.05.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-05-26
2,Karel Rüütli (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07
3,Ester Tuiksoo (09.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-09-04,2010-07-07
4,Tarmo Mänd (03.04.2007–12.03.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-03-04,2010-12-03
...,...,...,...,...,...
401,Marika Tuus-Laul,Eesti Keskerakonna fraktsioon,xiv,2019-04-04,2023-03-04
402,Jaak Valge,Eesti Konservatiivse Rahvaerakonna fraktsioon,xiv,2019-04-04,2023-03-04
403,Viktor Vassiljev,Eesti Keskerakonna fraktsioon,xiv,2019-04-04,2023-03-04
404,Aivar Viidik,Eesti Reformierakonna fraktsioon,xiv,2019-04-04,2023-03-04


## Do additional cleaning of names

In [28]:
#remove time info from names
df_factions_all['name_clean']=df_factions_all.name.apply(lambda x: x.split('(')[0].split(', ')[0].strip())
df_factions_all.head()

,name,faction,koosseis,from,to,name_clean
0,Mai Treial (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07,Mai Treial
1,Villu Reiljan (02.04.2007–26.05.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-05-26,Villu Reiljan
2,Karel Rüütli (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07,Karel Rüütli
3,Ester Tuiksoo (09.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-09-04,2010-07-07,Ester Tuiksoo
4,Tarmo Mänd (03.04.2007–12.03.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-03-04,2010-12-03,Tarmo Mänd


In [29]:
df_factions_all.name_clean.value_counts()

Enn Eesmaa          4
Heljo Pikhof        4
Sven Sester         4
Kalvi Kõva          4
Marko Mihkelson     4
                   ..
Tiit Tammsaar       1
Rannar Vassiljev    1
Remo Holsmer        1
Jaanus Tamkivi      1
Mart Võrklaev       1
Name: name_clean, Length: 270, dtype: int64

In [30]:
#check if have some funny characters
set(''.join(df_factions_all.name_clean.tolist()).lower())

{' ',
 '-',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'y',
 'z',
 'ä',
 'õ',
 'ö',
 'ü',
 'š'}

In [31]:
df_factions_all[df_factions_all.name_clean.str.contains('-')]

,name,faction,koosseis,from,to,name_clean
18,Mari-Ann Kelam,Isamaa ja Res Publica Liidu fraktsioon,xi,2007-02-04,2011-03-04,Mari-Ann Kelam
24,Liisa-Ly Pakosta,Isamaa ja Res Publica Liidu fraktsioon,xi,2007-02-04,2011-03-04,Liisa-Ly Pakosta
29,Ken-Marti Vaher,Isamaa ja Res Publica Liidu fraktsioon,xi,2007-02-04,2011-03-04,Ken-Marti Vaher
51,Katrin Karisma-Krumm,Eesti Reformierakonna fraktsioon,xi,2007-02-04,2011-03-04,Katrin Karisma-Krumm
65,Paul-Eerik Rummo,Eesti Reformierakonna fraktsioon,xi,2007-02-04,2011-03-04,Paul-Eerik Rummo
119,Liisa-Ly Pakosta,Isamaa ja Res Publica Liidu fraktsioon,xii,2011-04-04,2015-03-29,Liisa-Ly Pakosta
124,Helir-Valdor Seeder,Isamaa ja Res Publica Liidu fraktsioon,xii,2011-04-04,2015-03-29,Helir-Valdor Seeder
129,Ken-Marti Vaher,Isamaa ja Res Publica Liidu fraktsioon,xii,2011-04-04,2015-03-29,Ken-Marti Vaher
167,Tiina Lokk-Tramberg,Eesti Reformierakonna fraktsioon,xii,2011-04-04,2015-03-29,Tiina Lokk-Tramberg
177,Paul-Eerik Rummo,Eesti Reformierakonna fraktsioon,xii,2011-04-04,2015-03-29,Paul-Eerik Rummo


In [32]:
# members that have multiple factions in one koosseis - became MPs without faction (left faction/party)
df_factions_all[df_factions_all.duplicated(subset=['name_clean', 'koosseis'])]

,name,faction,koosseis,from,to,name_clean
101,Tarmo Mänd (alates 12.03.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-12-03,2011-03-04,Tarmo Mänd
102,Jaanus Marrandi (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-03-04,Jaanus Marrandi
103,Karel Rüütli (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-03-04,Karel Rüütli
105,Mai Treial (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-03-04,Mai Treial
106,Ester Tuiksoo (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-03-04,Ester Tuiksoo
107,Jaan Õunapuu (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-03-04,Jaan Õunapuu


## Save faction data

In [557]:
df_factions_all.to_csv('data/interim/factions.csv',index=False)

## Read stenos to add faction info

In [33]:
df_stenos=pd.read_csv('data/interim/stenos_cleaned.csv')
df_stenos.shape

(209949, 14)

In [34]:
df_stenos.head()

,heading,speaker,link_video,link_steno,index_pk,index_snd,text,year,month,day,time,seconds_from_start,video_link_wo_time,text_wo_punct
0,10:00 Istungi rakendamine,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=0,https://stenogrammid.riigikogu.ee/201911191000,PKP-25040,SND-559399,"Auväärt kolleegid, auväärt Riigikogu, tere hom...",2019,11,19,10:00,0,https://www.youtube.com/watch?v=GaFL2Zc4OMg,auväärt kolleegid auväärt riigikogu tere hommi...
1,1.\n 10:02 Eesti Keske...,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=125,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559400,Täna on meil päevakorras olulise tähtsusega ri...,2019,11,19,10:02,125,https://www.youtube.com/watch?v=GaFL2Zc4OMg,täna on meil päevakorras olulise tähtsusega ri...
2,1.\n 10:02 Eesti Keske...,Kersti Sarapuu,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=238,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559401,Lugupeetud istungi juhataja! Head kolleegid ja...,2019,11,19,10:02,238,https://www.youtube.com/watch?v=GaFL2Zc4OMg,lugupeetud istungi juhataja head kolleegid ja ...
3,1.\n 10:02 Eesti Keske...,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=...,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559402,"Aitäh! Ettekandjatele ütlen niipalju, et selle...",2019,11,19,10:02,1194,https://www.youtube.com/watch?v=GaFL2Zc4OMg,aitäh ettekandjatele ütlen niipalju et selles ...
4,1.\n 10:02 Eesti Keske...,Aivar Sõerd,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=...,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559403,"Aitäh, austatud juhataja! Austatud ettekandja!...",2019,11,19,10:02,1231,https://www.youtube.com/watch?v=GaFL2Zc4OMg,aitäh austatud juhataja austatud ettekandja ei...


## Add koosseis info

In [35]:
#create koossseis variable
df_stenos['date']=pd.to_datetime(df_stenos.year.astype(str)+'.'+df_stenos.month.astype(str)+'.'+df_stenos.day.astype(str))
df_stenos['koosseis']=np.where(df_stenos['date']<=pd.to_datetime(koosseis_end['xi']), 'xi', '')
df_stenos['koosseis']=np.where((df_stenos['date']<=pd.to_datetime(koosseis_end['xii']))&(df_stenos['date']>pd.to_datetime(koosseis_end['xi'])), 'xii', df_stenos['koosseis'])
df_stenos['koosseis']=np.where((df_stenos['date']<=pd.to_datetime(koosseis_end['xiii']))&(df_stenos['date']>pd.to_datetime(koosseis_end['xii'])), 'xiii', df_stenos['koosseis'])
df_stenos['koosseis']=np.where((df_stenos['date']<=pd.to_datetime(koosseis_end['xiv']))&(df_stenos['date']>pd.to_datetime(koosseis_end['xiii'])), 'xiv', df_stenos['koosseis'])

In [36]:
df_stenos.koosseis.value_counts()

xiii    72124
xiv     60994
xii     57013
xi      19818
Name: koosseis, dtype: int64

In [37]:
df_stenos.shape

(209949, 16)

## Add faction info

In [38]:
#this one takes time!
def get_stenos_factions(x):
    """adds faction member info to steno based on the time of Riigikogu koosseis and when somebody was in faction"""
    df_  = df_stenos[df_stenos.index_snd==x['index_snd']]
    if not df_['speaker'].tolist()[0] in df_factions_all.name_clean.tolist():
         return ''
    df_factions_all_sub=df_factions_all[(df_factions_all['to']>=df_.date.tolist()[0])&(df_factions_all['from']<=df_.date.tolist()[0])&(df_factions_all.name_clean==df_['speaker'].tolist()[0])]
    if df_factions_all_sub.shape[0]==0:
        return ''
    elif df_factions_all_sub.shape[0]==1:
        return df_factions_all_sub['faction'].tolist()[0]
    else:
        print(f'df shape {df_.shape} should be (0, n)')
        
%time stenos_factions=df_stenos.apply(get_stenos_factions, axis=1)

Wall time: 50min 6s


In [39]:
df_stenos['fation']=stenos_factions

In [40]:
df_stenos['fation'].value_counts()

                                                    135559
Eesti Keskerakonna fraktsioon                        16806
Eesti Reformierakonna fraktsioon                     15521
Eesti Konservatiivse Rahvaerakonna fraktsioon        11069
Sotsiaaldemokraatliku Erakonna fraktsioon            10050
Isamaa fraktsioon                                     6095
Isamaa ja Res Publica Liidu fraktsioon                5098
Eesti Vabaerakonna fraktsioon                         3385
Fraktsiooni mittekuuluvad saadikud                    2707
Fraktsioonidesse mittekuulunud Riigikogu liikmed      2446
Erakonna Eestimaa Rohelised fraktsioon                 486
Eestimaa Rahvaliidu fraktsioon                         411
Fraktsiooni mittekuuluvad Riigikogu liikmed            316
Name: fation, dtype: int64

## Save stenos

In [43]:
df_stenos.to_csv('data/interim/stenos_cleaned_faction.csv', index=False)